In [1]:
import numpy as np
from typing import Tuple

from tsc.utilities import Fermi, get_RPTS, get_KPTS
from tsc.basis_atoms import single_atom, slab, extract_atom_vectors
from tsc import config

In [2]:
# Get config variables
globals().update({k: v for k, v in vars(config.Config).items() if not k.startswith("__")})

In [1]:
# Call a function from basis_atoms.py to get the basis atoms array
atoms = single_atom()

# Extract the vectors from the called function
TPTS, atom_types, E_0, Ulcn, n_0, Beta, Vsc = extract_atom_vectors(atoms)

# Construct the lattice sites matrix
RPTS = get_RPTS(a_1, a_2, a_3, NCELLS)

NameError: name 'single_atom' is not defined

In [4]:
# Construct a hopping matrix - below we have the completely symmetrical case
# Get the number of different atom types based on the basis_atoms configuration
num_atom_types: int = np.unique(atom_types).shape[0]
# Construct a completely symmetrical case
hop_mat = np.ones((num_atom_types, num_atom_types))

In [ ]:
# Construct the k-mesh
KPTS = get_KPTS(a_1, a_2, a_3, N_x, N_y, N_z)

In [24]:
# For experiments:
atom_1 = single_atom()
atom_2 = single_atom(x=0.5, y = 0.5, z = 0.5, atom_type=2, E_0 = 1.0)
atom_3 = single_atom()
atoms = np.vstack((atom_1,atom_2,atom_3))

# Extract the vectors from the called function
TPTS, atom_types, E_0, Ulcn, n_0, Beta, Λ = extract_atom_vectors(atoms)

# Construct the lattice sites matrix
RPTS = get_RPTS(a_1, a_2, a_3, NCELLS)

# Construct the k-mesh
KPTS = get_KPTS(a_1, a_2, a_3, N_x, N_y, N_z)

In [56]:
# Get the number of different atom types based on the basis_atoms configuration
# Also re-encode the atom_types as 0, 1, 2, etc.
unique_atoms, atom_types = np.unique(atom_types, return_inverse=True)
num_atom_types: int = unique_atoms.shape[0]

# Construct a completely symmetrical case
hop_mat = np.ones((num_atom_types, num_atom_types))

In [26]:
NUMT = atoms.shape[0]

In [54]:
# subroutine HOPPS
PREFACTORS = np.zeros((num_atom_types,num_atom_types))

NNDIS = 1000*np.ones((num_atom_types,num_atom_types))
for itype in range(num_atom_types):
    for jtype in range(num_atom_types):
        
        for i in range(NUMT):
            for j in range(NUMT):
                
                if (atom_types[i] == itype) and (atom_types[j] == jtype):
                    ttprime = TPTS[j] - TPTS[i]
                    for rpoint in RPTS:
                        addd = np.linalg.norm(rpoint + ttprime)

                        if (NNDIS[itype,jtype] > addd) and (addd > 1e-4):
                            NNDIS[itype,jtype] = addd
                            
        expon = np.exp(-NNDIS[itype,jtype]/R_0)
        PREFACTORS[itype,jtype] = hop_mat[itype,jtype]/expon


In [55]:
PREFACTORS

array([[2.71828183, 2.37744268],
       [2.37744268, 2.71828183]])

In [39]:
# NEIGHBNUM calculator
NEIGHBNUM = np.zeros((NUMT), dtype=int)

for i in range(NUMT):
    for j in range(NUMT):

        ttprime = TPTS[j] - TPTS[i]
        for rpoint in RPTS:
            addd = np.linalg.norm(rpoint + ttprime)
            if (addd < R_max + 1e-5) and (addd > 1e-5):
                NEIGHBNUM[i] += 1

In [50]:
# JTYPE and RCONNECT calculator
MAXNEIGHB = NEIGHBNUM.max()
JTYPE = np.zeros((NUMT, MAXNEIGHB), dtype=int)
RCONNECT = np.zeros((NUMT, MAXNEIGHB,3))

for i in range(NUMT):
    rcheck = 0
    for j in range(NUMT):

        ttprime = TPTS[j] - TPTS[i]
        for rpoint in RPTS:
            addd = np.linalg.norm(rpoint + ttprime)
            if (addd < R_max + 1e-5) and (addd > 1e-5):
                RCONNECT[i,rcheck,:] = rpoint + ttprime
                JTYPE[i,rcheck] = j
                rcheck += 1

In [59]:
# Hopping elements calculations
expons = np.zeros((NUMK,MAXNEIGHB,NUMT), dtype=complex)
hoppvals = np.zeros((MAXNEIGHB,NUMT))

for kcounter in range(NUMK):
    kpoint = KPTS[:,kcounter]
    for i in range(NUMT):
        for j in range(NUMT):

            rpoint = RCONNECT[:,j,i]
            expons[kcounter,j,i] = np.exp(-1j*np.dot(kpoint,rpoint))

            JATOM = JTYPE[j,i].astype(int)
            llambda = PREFACTORS[atom_types[i],atom_types[JATOM]]

            hoppvals[j,i] = -llambda*np.exp(-np.linalg.norm(rpoint)/R_0)

In [60]:
hoppvals

array([[-0.13533528, -0.17692121],
       [-0.13533528, -0.17692121],
       [ 0.        ,  0.        ],
       [ 0.        ,  0.        ],
       [ 0.        ,  0.        ],
       [ 0.        ,  0.        ],
       [ 0.        ,  0.        ],
       [ 0.        ,  0.        ],
       [ 0.        ,  0.        ],
       [ 0.        ,  0.        ],
       [ 0.        ,  0.        ],
       [ 0.        ,  0.        ],
       [ 0.        ,  0.        ],
       [ 0.        ,  0.        ]])

In [47]:
atom_types

array([0, 1])

In [66]:
RCONNECT[:,:,0]

array([[-1. ,  0. ,  0. ,  0. ,  0. ,  1. , -0.5, -0.5, -0.5, -0.5,  0.5,
         0.5,  0.5,  0.5],
       [ 0. , -1. ,  0. ,  0. ,  1. ,  0. , -0.5, -0.5,  0.5,  0.5, -0.5,
        -0.5,  0.5,  0.5],
       [ 0. ,  0. , -1. ,  1. ,  0. ,  0. , -0.5,  0.5, -0.5,  0.5, -0.5,
         0.5, -0.5,  0.5]])

In [67]:
RCONNECT.shape

(3, 14, 2)

In [ ]:
# Construct the k-mesh
KPTS = get_KPTS()

In [27]:
# NEEDS FIXING
def get_NumDen(EIGENVALUES: np.ndarray, EIGENVECTORS: np.ndarray, NUMT: int, NUMK: int, 
            PI: float, NUME: int, lorentzbroad: float, metalorno: int) -> None:
    
    numdensityperatom = np.zeros((NUMT + 1, NUME))
    numdensity = np.zeros((2, NUME))
    
    energyintervals = (np.max(EIGENVALUES) - np.min(EIGENVALUES)) / (NUME - 1)
    
    # These are the energies E
    energies = np.min(EIGENVALUES) + energyintervals * np.arange(NUME)
    numdensityperatom[0, :] = energies
    numdensity[0, :] = energies
    numdensity[1, :] = 0.0
    
    # Calculation of the DoS PER ATOM
    term1 = np.abs(EIGENVECTORS[:NUMT, :])**2
    term2 = np.abs(EIGENVECTORS[NUMT:, :])**2
    terms_sum = term1 + term2
    
    # Reshape terms_sum to have an extra dimension for NUME
    terms_sum_reshaped = terms_sum[:, np.newaxis, :]
    
    # Reshape diff_square to have an extra dimension for NUMT
    diff_square = (energies[:, np.newaxis] - EIGENVALUES)**2
    diff_square_reshaped = diff_square[np.newaxis, :, :]
    
    # Now, both arrays have a shape of (NUMT, NUME, 4*NUMT*NUMK)
    dos_terms = (lorentzbroad / PI) * terms_sum_reshaped / (diff_square_reshaped + lorentzbroad**2)
    
    numdensityperatom[1:, :] = np.sum(dos_terms, axis=2)
    
    # Normalization
    numdensityperatom[1:, :] /= NUMK
    
    # File writing
    if metalorno == 0:
        np.savetxt('numdensityperatom2.txt', numdensityperatom.T, fmt='%17.8f')
        numdensity[1, :] = np.mean(numdensityperatom[1:, :], axis=0)
        np.savetxt('numdensity2.txt', numdensity.T, fmt='%17.8f')
    elif metalorno == 1:
        np.savetxt('metalnumdensityperatom2.txt', numdensityperatom.T, fmt='%17.8f')
        numdensity[1, :] = np.mean(numdensityperatom[1:, :], axis=0)
        np.savetxt('metalnumdensity2.txt', numdensity.T, fmt='%17.8f')


In [70]:
vectorized_result

array([10000])

In [71]:
original_result

array([10000., 10000., 10000., 10000., 10000., 10000., 10000., 10000.,
       10000., 10000., 10000., 10000., 10000., 10000., 10000., 10000.,
       10000., 10000., 10000., 10000., 10000., 10000., 10000., 10000.,
       10000., 10000., 10000., 10000., 10000., 10000., 10000., 10000.,
       10000., 10000., 10000., 10000., 10000., 10000., 10000., 10000.,
       10000., 10000., 10000., 10000., 10000., 10000., 10000., 10000.,
       10000., 10000., 10000., 10000., 10000., 10000., 10000., 10000.,
       10000., 10000., 10000., 10000., 10000., 10000., 10000., 10000.,
       10000., 10000., 10000., 10000., 10000., 10000., 10000., 10000.,
       10000., 10000., 10000., 10000., 10000., 10000., 10000., 10000.,
       10000., 10000., 10000., 10000., 10000., 10000., 10000., 10000.,
       10000., 10000., 10000., 10000., 10000., 10000., 10000., 10000.,
       10000., 10000., 10000., 10000.])